# Question 1: Distance Matrix Calculation with solution

In [11]:
import pandas as pd

def calculate_distance_matrix(csv_file):
    # Load the dataset into a DataFrame
    df = pd.read_csv(csv_file)

    # Initialize an empty matrix with zeros
    unique_ids = sorted(set(df['id_start'].unique()) | set(df['id_end'].unique()))
    distance_matrix = pd.DataFrame(0, index=unique_ids, columns=unique_ids)

    # Iterate over rows in the dataset and update the distance matrix
    for row in df.itertuples(index=False):
        start_id = row.id_start
        end_id = row.id_end
        distance = row.distance

        # Update the distance matrix symmetrically
        distance_matrix.at[start_id, end_id] += distance
        distance_matrix.at[end_id, start_id] += distance

    # Calculate cumulative distances along known routes
    for i in unique_ids:
        for j in unique_ids:
            for k in unique_ids:
                if distance_matrix.at[i, j] > 0 and distance_matrix.at[j, k] > 0:
                    cumulative_distance = distance_matrix.at[i, j] + distance_matrix.at[j, k]
                    if distance_matrix.at[i, k] == 0:
                        distance_matrix.at[i, k] = cumulative_distance
                        distance_matrix.at[k, i] = cumulative_distance

    # Set diagonal values to 0
    distance_matrix.values[[range(len(unique_ids))]*2] = 0

    return distance_matrix

# Example usage'‪
distance_matrix = calculate_distance_matrix(r"C:\Users\user\Desktop\PowerBi\excel assesment\dataset-3 three")
print(distance_matrix)


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400     19.4      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7     19.4     20.2     55.6     77.3     88.4    104.0   
1001404     29.9     20.2     59.8     16.0     97.5    108.6    124.2   
1001406     45.9     55.6     16.0     91.8     21.7    124.6    140.2   
1001408     67.6     77.3     97.5     21.7    135.2     11.1    161.9   
1001410     78.7     88.4    108.6    124.6     11.1    157.4     15.6   
1001412     94.3    104.0    124.2    140.2    161.9     15.6    188.6   
1001414    112.5    122.2    142.4    158.4    180.1    191.2     18.2   
1001416    125.7    135.4    155.6    171.6    193.3    204.4    220.0   
1001418    139.3    149.0    169.2    185.2    206.9    218.0    233.6   
1001420    152.2    161.9    182.1    198.1    219.8    230.9    246.5   
1001422    161.8    171.5    191.7    207.7    229.4    240.5    256.1   
1001424    173.2    182.9    203.1    

# Question 2: Unroll Distance Matrix with solution

In [12]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Initialize an empty list to store unrolled distance data
    unrolled_data = []

    # Iterate over rows and columns of the distance matrix
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            # Exclude same id_start to id_end
            if id_start != id_end:
                distance = distance_matrix.at[id_start, id_end]
                unrolled_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

    # Create a DataFrame from the unrolled data
    unrolled_df = pd.DataFrame(unrolled_data)

    return unrolled_df

# Example usage with the distance_matrix generated from Question 1
unrolled_distance_df = unroll_distance_matrix(distance_matrix)
print(unrolled_distance_df)


      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1004356  1001470     756.6
1802   1004356  1001472     772.6
1803   1004356  1001488       4.0
1804   1004356  1004354       2.0
1805   1004356  1004355     560.4

[1806 rows x 3 columns]


# Question 3: Finding IDs within Percentage Threshold with solutions

In [13]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(df, reference_value):
    # Filter DataFrame based on the reference value
    reference_data = df[df['id_start'] == reference_value]

    # Calculate the average distance for the reference value
    average_distance = reference_data['distance'].mean()

    # Calculate the lower and upper bounds of the 10% threshold
    lower_bound = average_distance - 0.1 * average_distance
    upper_bound = average_distance + 0.1 * average_distance

    # Filter DataFrame based on the 10% threshold
    within_threshold = df[(df['distance'] >= lower_bound) & (df['distance'] <= upper_bound)]

    # Get unique values from the id_start column and sort them
    result_ids = sorted(within_threshold['id_start'].unique())

    return result_ids

# Example usage with the unrolled_distance_df generated from Question 2
reference_value = 1001400  # Replace with the desired reference value
result_ids = find_ids_within_ten_percentage_threshold(unrolled_distance_df, reference_value)
print(result_ids)


[1001400, 1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001416, 1001418, 1001420, 1001422, 1001424, 1001426, 1001428, 1001430, 1001432, 1001434, 1001436, 1001437, 1001438, 1001440, 1001442, 1001444, 1001488, 1004354, 1004355, 1004356]


# Question 4: Calculate Toll Rate with solution

In [14]:
import pandas as pd

def calculate_toll_rate(df):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Add columns for each vehicle type with their respective toll rates
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df[vehicle_type] = df['distance'] * rate_coefficient

    return df

# Example usage with the unrolled_distance_df generated from Question 2
toll_rate_df = calculate_toll_rate(unrolled_distance_df)
print(toll_rate_df)


      id_start   id_end  distance    moto     car       rv      bus    truck
0      1001400  1001402       9.7    7.76   11.64    14.55    21.34    34.92
1      1001400  1001404      29.9   23.92   35.88    44.85    65.78   107.64
2      1001400  1001406      45.9   36.72   55.08    68.85   100.98   165.24
3      1001400  1001408      67.6   54.08   81.12   101.40   148.72   243.36
4      1001400  1001410      78.7   62.96   94.44   118.05   173.14   283.32
...        ...      ...       ...     ...     ...      ...      ...      ...
1801   1004356  1001470     756.6  605.28  907.92  1134.90  1664.52  2723.76
1802   1004356  1001472     772.6  618.08  927.12  1158.90  1699.72  2781.36
1803   1004356  1001488       4.0    3.20    4.80     6.00     8.80    14.40
1804   1004356  1004354       2.0    1.60    2.40     3.00     4.40     7.20
1805   1004356  1004355     560.4  448.32  672.48   840.60  1232.88  2017.44

[1806 rows x 8 columns]


# Question 5: Calculate Time-Based Toll Rates

In [15]:
import pandas as pd
from datetime import datetime, time, timedelta

def calculate_time_based_toll_rates(df):
    # Define time ranges and discount factors
    time_ranges_weekdays = [(time(0, 0, 0), time(10, 0, 0)),
                            (time(10, 0, 0), time(18, 0, 0)),
                            (time(18, 0, 0), time(23, 59, 59))]

    time_ranges_weekends = [(time(0, 0, 0), time(23, 59, 59))]

    discount_factors_weekdays = [0.8, 1.2, 0.8]
    discount_factor_weekends = 0.7

    # Create a list to store time-based toll rates
    time_based_toll_rates = []

    # Iterate over unique (id_start, id_end) pairs
    for pair in df[['id_start', 'id_end']].drop_duplicates().itertuples(index=False):
        for day in range(7):  # Iterate over days from Monday to Sunday
            for time_range, discount_factor in zip(time_ranges_weekdays, discount_factors_weekdays):
                start_time = datetime.combine(datetime(2023, 1, 1), time_range[0])
                end_time = datetime.combine(datetime(2023, 1, 1), time_range[1])

                start_time += timedelta(days=day)
                end_time += timedelta(days=day)

                time_based_toll_rates.append({
                    'id_start': pair.id_start,
                    'id_end': pair.id_end,
                    'start_day': datetime(2023, 1, 1 + day).strftime('%A'),
                    'end_day': datetime(2023, 1, 1 + day).strftime('%A'),
                    'start_time': start_time.time(),
                    'end_time': end_time.time(),
                    'moto': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['moto'].values[0] * discount_factor,
                    'car': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['car'].values[0] * discount_factor,
                    'rv': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['rv'].values[0] * discount_factor,
                    'bus': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['bus'].values[0] * discount_factor,
                    'truck': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['truck'].values[0] * discount_factor,
                })

            for time_range in time_ranges_weekends:
                start_time = datetime.combine(datetime(2023, 1, 1), time_range[0])
                end_time = datetime.combine(datetime(2023, 1, 1), time_range[1])

                start_time += timedelta(days=day)
                end_time += timedelta(days=day)

                time_based_toll_rates.append({
                    'id_start': pair.id_start,
                    'id_end': pair.id_end,
                    'start_day': datetime(2023, 1, 1 + day).strftime('%A'),
                    'end_day': datetime(2023, 1, 1 + day).strftime('%A'),
                    'start_time': start_time.time(),
                    'end_time': end_time.time(),
                    'moto': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['moto'].values[0] * discount_factor_weekends,
                    'car': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['car'].values[0] * discount_factor_weekends,
                    'rv': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['rv'].values[0] * discount_factor_weekends,
                    'bus': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['bus'].values[0] * discount_factor_weekends,
                    'truck': df[(df['id_start'] == pair.id_start) & (df['id_end'] == pair.id_end)]['truck'].values[0] * discount_factor_weekends,
                })

    # Create a DataFrame from the time-based toll rates
    time_based_toll_rates_df = pd.DataFrame(time_based_toll_rates)

    return time_based_toll_rates_df

# Example usage with the toll_rate_df generated from Question 4
time_based_toll_rates_df = calculate_time_based_toll_rates(toll_rate_df)
print(time_based_toll_rates_df)


       id_start   id_end start_day   end_day start_time  end_time     moto  \
0       1001400  1001402    Sunday    Sunday   00:00:00  10:00:00    6.208   
1       1001400  1001402    Sunday    Sunday   10:00:00  18:00:00    9.312   
2       1001400  1001402    Sunday    Sunday   18:00:00  23:59:59    6.208   
3       1001400  1001402    Sunday    Sunday   00:00:00  23:59:59    5.432   
4       1001400  1001402    Monday    Monday   00:00:00  10:00:00    6.208   
...         ...      ...       ...       ...        ...       ...      ...   
50563   1004356  1004355    Friday    Friday   00:00:00  23:59:59  313.824   
50564   1004356  1004355  Saturday  Saturday   00:00:00  10:00:00  358.656   
50565   1004356  1004355  Saturday  Saturday   10:00:00  18:00:00  537.984   
50566   1004356  1004355  Saturday  Saturday   18:00:00  23:59:59  358.656   
50567   1004356  1004355  Saturday  Saturday   00:00:00  23:59:59  313.824   

           car        rv       bus     truck  
0        9.312  